In [34]:
from sidecar import Sidecar
from ipywidgets import IntSlider, Layout
from ipyleaflet import Map, basemaps, Marker, projections
import xarray as xr
import xarray_leaflet
import odc.ui
# import hvplot

## Load data

In [32]:
data = xr.open_dataset('/g/data/jk72/MIZ/Bremen/netcdf/asi-AMSR2-s6250-2012-2022.nc')

# get monthly averages

monthly_average = data.groupby('time.month').mean('time')

monthly_average = monthly_average.rename_dims({'month':'time'})

In [33]:
jan = monthly_average['conc'].isel(time=0)

## Create tracker interface

In [39]:
# South Polar sterographic
spsLayout=Layout(width='800px', height='800px')

m = Map(center=(-90, 0),
        zoom=0,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

sc = Sidecar(title='Sidecar Output')


with sc:
    display(m)

ValueError: Expect RGB xr.DataArray

In [ ]:
# Expects an RGB input, not a single band like conc. Needs modifying to work.

# img_layer = odc.ui.mk_image_overlay(
#     jan,
#     clamp=2000,  # 2000 -- brightest pixel level
#     bands=['conc'],
#     fmt='png')   # "jpeg" is another option

# Add image layer to a map we created earlier
# m.add_layer(img_layer)

In [6]:
center = (-70, 70)

m = Map(center=center, zoom=5)

marker = Marker(location=center, draggable=True)
m.add_layer(marker);


# Now that the marker is on the Map, you can drag it with your mouse,
# it will automatically update the `marker.location` attribute in Python

# You can also update the marker location from Python, that will update the
# marker location on the Map:
marker.location = (50, 356)

sl = IntSlider(description='Some slider')
with sc:
#     display(sl)
    display(m)